In [ ]:
import pandas as pd
import numpy as np
import os

In [2]:
print("Loading datasets...")
# Load datasets
dataset1 = pd.read_csv('in_sample.csv')
dataset2 = pd.read_csv('out_sample.csv')
in_sample_risk_free = pd.read_csv('in_sample_risk_free.csv')
out_sample_risk_free = pd.read_csv('out_sample_risk_free.csv')
print("Datasets loaded.")

Loading datasets...


C:\Users\amaan\AppData\Local\Temp\ipykernel_16048\1574468555.py:3: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset1 = pd.read_csv('in_sample.csv')


Datasets loaded.


In [3]:
print("Displaying dataset information of stocks data from 1st Jan 2000 to 31st Dec 2000...\n")
print(dataset1.info())

Displaying dataset information of stocks data from 1st Jan 2000 to 31st Dec 2000...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9924981 entries, 0 to 9924980
Data columns (total 22 columns):
 #   Column         Dtype  
---  ------         -----  
 0   PERMNO         int64  
 1   HdrCUSIP       object 
 2   CUSIP          object 
 3   PrimaryExch    object 
 4   ShareType      object 
 5   Ticker         object 
 6   TradingSymbol  object 
 7   PERMCO         int64  
 8   SICCD          int64  
 9   DlyCalDt       object 
 10  DlyPrc         float64
 11  DlyCap         float64
 12  DlyRet         float64
 13  DlyRetx        float64
 14  DlyRetI        float64
 15  DlyVol         float64
 16  DlyClose       float64
 17  DlyLow         float64
 18  DlyHigh        float64
 19  DlyNumTrd      float64
 20  DlyPrcVol      float64
 21  ShrOut         float64
dtypes: float64(12), int64(3), object(7)
memory usage: 1.6+ GB
None


In [4]:
print("Displaying dataset information of stocks data from 1st Jan 2016 to 31st Dec 2024...\n")
print(dataset2.info())

Displaying dataset information of stocks data from 1st Jan 2016 to 31st Dec 2024...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8773888 entries, 0 to 8773887
Data columns (total 22 columns):
 #   Column         Dtype  
---  ------         -----  
 0   PERMNO         int64  
 1   HdrCUSIP       object 
 2   CUSIP          object 
 3   PrimaryExch    object 
 4   ShareType      object 
 5   Ticker         object 
 6   TradingSymbol  object 
 7   PERMCO         int64  
 8   SICCD          int64  
 9   DlyCalDt       object 
 10  DlyPrc         float64
 11  DlyCap         float64
 12  DlyRet         float64
 13  DlyRetx        float64
 14  DlyRetI        float64
 15  DlyVol         float64
 16  DlyClose       float64
 17  DlyLow         float64
 18  DlyHigh        float64
 19  DlyNumTrd      float64
 20  DlyPrcVol      float64
 21  ShrOut         float64
dtypes: float64(12), int64(3), object(7)
memory usage: 1.4+ GB
None


In [5]:
# Convert the DlyCalDt columns to DlyCalDttime format
print("Converting DlyCalDt columns to DlyCalDttime format...\n")

dataset1['DlyCalDt'] = pd.to_datetime(dataset1['DlyCalDt'])
dataset2['DlyCalDt'] = pd.to_datetime(dataset2['DlyCalDt'])
in_sample_risk_free['CALDT'] = pd.to_datetime(in_sample_risk_free['CALDT'])
out_sample_risk_free['CALDT'] = pd.to_datetime(out_sample_risk_free['CALDT'])
    
in_sample_risk_free.rename(columns={"CALDT": "DlyCalDt"}, inplace=True)
in_sample_risk_free.sort_values(by='DlyCalDt', inplace=True)

out_sample_risk_free.rename(columns={"CALDT": "DlyCalDt"}, inplace=True)
out_sample_risk_free.sort_values(by='DlyCalDt', inplace=True)

Converting DlyCalDt columns to DlyCalDttime format...



In [6]:
print("Merging risk-free rate with the datasets...\n")


# Function to merge risk-free rate with the dataset
def merge_risk_free(dataset, risk_free):
    
    # Since there are some missing values in the risk-free rate, we forward fill them.
    risk_free['TDYLD'] = risk_free['TDYLD'].ffill()

    # Since some dates have multiple entries, we take the mean of the risk-free rate for each date.
    risk_free = risk_free.groupby('DlyCalDt')['TDYLD'].mean().reset_index()
    
    # Merge the risk-free rate with the dataset on the date column.
    # We use a left join to keep all the rows in the dataset.
    dataset = pd.merge(dataset, risk_free[["DlyCalDt", "TDYLD"]], how="left", on="DlyCalDt")

    return dataset

dataset1 = merge_risk_free(dataset1, in_sample_risk_free)
dataset2 = merge_risk_free(dataset2, out_sample_risk_free)
#risk_free

Merging risk-free rate with the datasets...



In [7]:
print("Calculating the market cap...\n")

# Adding market cap column by calculating the product of daily close price and shares outstanding

def calculate_market_cap(dataframe):

    # Calculate market cap as the product of daily close price and shares outstanding
    dataframe['MktCap'] = dataframe['DlyClose'] * dataframe['ShrOut']   

    # Sort the dataframe by market cap in descending order
    dataframe.sort_values(by='MktCap', ascending=False, inplace=True)
    
    return dataframe

dataset1 = calculate_market_cap(dataset1)
dataset2 = calculate_market_cap(dataset2)

Calculating the market cap...



In [8]:
print("Calculating the daily excess return by adjusting the risk-free rate...\n")

# Adding a column for the daily excess return
def calculate_excess_return(dataframe):
    
    # Calculate daily excess return as the difference between daily return and risk-free rate
    dataframe['ExcessReturn'] = dataframe['DlyRet'] - dataframe['TDYLD']
   
    return dataframe

dataset1 = calculate_excess_return(dataset1)
dataset2 = calculate_excess_return(dataset2)    

Calculating the daily excess return by adjusting the risk-free rate...



In [9]:
# Dropping null values and unwanted columns 

def drop_null_values(dataframe):
    # Dropping unwanted columns
    dataframe.drop(columns='DlyNumTrd', inplace=True)
    dataframe.drop(columns='TradingSymbol', inplace = True)
    dataframe.drop(columns='PrimaryExch', inplace = True)
    dataframe.drop(columns='ShareType', inplace = True)
    dataframe.drop(columns='DlyRetI', inplace = True)
    dataframe.drop(columns='HdrCUSIP', inplace = True)
    
    # Dropping rows with null values in market cap, daily return, and daily volume columns
    dataframe.dropna(subset=['MktCap'], inplace=True)
    dataframe.dropna(subset=['DlyRet'], inplace=True)
    dataframe.dropna(subset=['DlyVol'], inplace=True)
    dataframe.dropna(subset=['ExcessReturn'], inplace = True)

    return dataframe

dataset1 = drop_null_values(dataset1)
dataset2 = drop_null_values(dataset2)

dataset1.drop_duplicates(subset=['PERMNO', 'DlyCalDt'], keep='first', inplace=True, ignore_index=True)
dataset2.drop_duplicates(subset=['PERMNO', 'DlyCalDt'], keep='first', inplace=True, ignore_index=True)

Data is cleaned and null values are dropped.

In [10]:
# Remove stocks with incomplete data (all stocks should be traded for all trading days)

def filter_all_trading_days(df):

    common_trading_days = np.sort(df["DlyCalDt"].unique())
    expected_days = len(common_trading_days)
    # Build the Common Trading Calendar
    print(f"Number of trading days: {expected_days}")
    # Group the data by stock (PERMNO) and count distinct trading days

    stock_counts = df.groupby("PERMNO")["DlyCalDt"].nunique().reset_index()
        
    # Identify stocks that have complete data
    valid_stocks = stock_counts[stock_counts["DlyCalDt"] == expected_days]["PERMNO"].tolist()
    print(f"Number of stocks with complete data for all trading days: {len(valid_stocks)}")

    # Report stocks removed
    removed_stocks = stock_counts[stock_counts["DlyCalDt"] < expected_days]["PERMNO"].tolist()
    print(f"Number of stocks removed due to incomplete data: {len(removed_stocks)}")

    # Filter the main DataFrame to keep only the stocks with complete data
    df_clean = df[df["PERMNO"].isin(valid_stocks)].copy()
    
    #print(df_clean.head())
    #print(df_clean.shape[0])
    return df_clean

dataset1 = filter_all_trading_days(dataset1)
dataset2 = filter_all_trading_days(dataset2)

# Remove stocks that are not present in both datasets
# Find common PERMNOs in both datasets
common_permnos = set(dataset1['PERMNO']).intersection(set(dataset2['PERMNO'])) 

print(f"Number of common stocks: {len(common_permnos)}")
dataset1 = dataset1[dataset1['PERMNO'].isin(common_permnos)]
dataset2 = dataset2[dataset2['PERMNO'].isin(common_permnos)]

Number of trading days: 3991
Number of stocks with complete data for all trading days: 368
Number of stocks removed due to incomplete data: 5429
Number of trading days: 2248
Number of stocks with complete data for all trading days: 1604
Number of stocks removed due to incomplete data: 5944
Number of common stocks: 275


In [11]:
# Step 1: Clean both datasets
dataset1_clean = filter_all_trading_days(dataset1)
dataset2_clean = filter_all_trading_days(dataset2)

# Step 2: Get top 50 PERMNOs from in-sample data (only)
top_50_permnos = (
    dataset1_clean.sort_values(by='MktCap', ascending=False)['PERMNO']
    .drop_duplicates()
    .iloc[:50]
    .tolist()
)

# Step 3: Filter both datasets using the same top 50
in_sample_df = dataset1_clean[dataset1_clean['PERMNO'].isin(top_50_permnos)].copy()
out_sample_df = dataset2_clean[dataset2_clean['PERMNO'].isin(top_50_permnos)].copy()

# Sanity check:
#print(sorted(in_sample_df['PERMNO'].unique()) == sorted(out_sample_df['PERMNO'].unique()))

Number of trading days: 3991
Number of stocks with complete data for all trading days: 275
Number of stocks removed due to incomplete data: 0
Number of trading days: 2248
Number of stocks with complete data for all trading days: 275
Number of stocks removed due to incomplete data: 0


In [12]:
in_sample_df

,PERMNO,CUSIP,Ticker,PERMCO,SICCD,DlyCalDt,DlyPrc,DlyCap,DlyRet,DlyRetx,DlyVol,DlyClose,DlyLow,DlyHigh,DlyPrcVol,ShrOut,TDYLD,MktCap,ExcessReturn
0,38703,94974610,WFC,21305,6021,2015-07-22,58.52000,3.013315e+08,0.009836,0.009836,14871061.0,58.52000,57.99000,58.59000,870254489.7,5149205.0,0.000001,3.013315e+08,0.009835
1,38703,94974610,WFC,21305,6021,2015-07-23,58.21000,2.997352e+08,-0.005297,-0.005297,13582244.0,58.21000,58.06990,58.76500,790622423.2,5149205.0,0.000002,2.997352e+08,-0.005299
2,38703,94974610,WFC,21305,6021,2015-07-16,58.17000,2.995293e+08,0.006750,0.006750,13194493.0,58.17000,57.98000,58.35000,767523657.8,5149205.0,0.000001,2.995293e+08,0.006749
3,38703,94974610,WFC,21305,6021,2015-07-30,58.15000,2.994263e+08,0.003278,0.003278,9864823.0,58.15000,57.72000,58.19000,573639457.5,5149205.0,0.000002,2.994263e+08,0.003276
4,38703,94974610,WFC,21305,6021,2015-07-20,58.05000,2.989114e+08,0.001899,0.001899,10084859.0,58.05000,57.88000,58.29000,585426065.0,5149205.0,0.000002,2.989114e+08,0.001897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6733505,86455,81369Y60,XLF,34957,6726,2000-09-22,28.09375,7.444844e+04,0.019274,0.019274,91400.0,28.09375,27.43750,28.18750,2567768.8,2650.0,0.000166,7.444844e+04,0.019108
6739592,86455,81369Y60,XLF,34957,6726,2000-09-01,28.00000,7.420000e+04,-0.016465,-0.016465,751200.0,28.00000,28.00000,28.50000,21033600.0,2650.0,0.000169,7.420000e+04,-0.016634
6747534,86455,81369Y60,XLF,34957,6726,2000-09-20,27.87500,7.386875e+04,-0.014365,-0.014365,179500.0,27.87500,27.34375,28.15625,5003562.5,2650.0,0.000167,7.386875e+04,-0.014532
6759329,86455,81369Y60,XLF,34957,6726,2000-09-18,27.68750,7.337188e+04,-0.033279,-0.033279,430300.0,27.68750,27.57813,28.35938,11913931.3,2650.0,0.000166,7.337188e+04,-0.033445


In [13]:
out_sample_df

,PERMNO,CUSIP,Ticker,PERMCO,SICCD,DlyCalDt,DlyPrc,DlyCap,DlyRet,DlyRetx,DlyVol,DlyClose,DlyLow,DlyHigh,DlyPrcVol,ShrOut,TDYLD,MktCap,ExcessReturn
0,47896,46625H10,JPM,20436,6021,2024-11-25,250.29,704651448.6,0.007001,0.007001,10295350.0,250.29,249.0600,254.31,2.576823e+09,2815340.0,0.000121,704651448.6,0.006880
1,47896,46625H10,JPM,20436,6021,2024-11-26,249.97,703750539.8,-0.001279,-0.001279,6212142.0,249.97,248.3400,251.00,1.552849e+09,2815340.0,0.000121,703750539.8,-0.001400
2,47896,46625H10,JPM,20436,6021,2024-11-27,249.79,703243778.6,-0.000720,-0.000720,5472265.0,249.79,248.7710,251.19,1.366917e+09,2815340.0,0.000118,703243778.6,-0.000838
3,47896,46625H10,JPM,20436,6021,2024-11-29,249.72,703046704.8,-0.000280,-0.000280,5494825.0,249.72,249.4450,251.77,1.372168e+09,2815340.0,0.000116,703046704.8,-0.000396
4,47896,46625H10,JPM,20436,6021,2024-11-22,248.55,699752757.0,0.015485,0.015485,7997307.0,248.55,243.0347,249.15,1.987731e+09,2815340.0,0.000117,699752757.0,0.015368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1380832,66325,78442P10,SLM,6412,6141,2016-01-15,5.42,2310546.0,-0.009141,-0.009141,3096118.0,5.42,5.2700,5.51,1.678096e+07,426300.0,0.000007,2310546.0,-0.009148
1382257,66325,78442P10,SLM,6412,6141,2016-01-20,5.41,2306283.0,0.005576,0.005576,4161726.0,5.41,5.0900,5.47,2.251494e+07,426300.0,0.000007,2306283.0,0.005569
1383615,66325,78442P10,SLM,6412,6141,2016-02-09,5.40,2302106.4,-0.065744,-0.065744,5968411.0,5.40,5.3400,5.91,3.222942e+07,426316.0,0.000008,2302106.4,-0.065752
1386591,66325,78442P10,SLM,6412,6141,2016-01-19,5.38,2293494.0,-0.007380,-0.007380,5083158.0,5.38,5.2500,5.60,2.734739e+07,426300.0,0.000007,2293494.0,-0.007387


In [ ]:
# Saving the cleaned datasets to a CSV file

os.makedirs('Cleaned Datasets', exist_ok=True)
os.chdir('Cleaned Datasets')
in_sample_df.to_csv('in_sample_cleaned.csv', index=False)
out_sample_df.to_csv('out_sample_cleaned.csv', index=False)